In [14]:
import argparse
import glob
import json
import multiprocessing
import os
import random
import re
from importlib import import_module
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import f1_score

from dataset import MaskBaseDataset
from loss import create_criterion

import loss
import utils
from pprint import pprint

def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)


def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']


def grid_image(np_images, gts, preds, n=16, shuffle=False):
    batch_size = np_images.shape[0]
    assert n <= batch_size

    choices = random.choices(range(batch_size), k=n) if shuffle else list(range(n))
    figure = plt.figure(figsize=(12, 18 + 2))  # cautions: hardcoded, 이미지 크기에 따라 figsize 를 조정해야 할 수 있습니다. T.T
    plt.subplots_adjust(top=0.8)               # cautions: hardcoded, 이미지 크기에 따라 top 를 조정해야 할 수 있습니다. T.T
    n_grid = np.ceil(n ** 0.5)
    tasks = ["mask", "gender", "age"]
    for idx, choice in enumerate(choices):
        gt = gts[choice].item()
        pred = preds[choice].item()
        image = np_images[choice]
        # title = f"gt: {gt}, pred: {pred}"
        gt_decoded_labels = MaskBaseDataset.decode_multi_class(gt)
        pred_decoded_labels = MaskBaseDataset.decode_multi_class(pred)
        title = "\n".join([
            f"{task} - gt: {gt_label}, pred: {pred_label}"
            for gt_label, pred_label, task
            in zip(gt_decoded_labels, pred_decoded_labels, tasks)
        ])

        plt.subplot(n_grid, n_grid, idx + 1, title=title)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(image, cmap=plt.cm.binary)

    return figure


def increment_path(path, exist_ok=False):
    """ Automatically increment path, i.e. runs/exp --> runs/exp0, runs/exp1 etc.

    Args:
        path (str or pathlib.Path): f"{model_dir}/{args.name}".
        exist_ok (bool): whether increment path (increment if False).
    """
    path = Path(path)
    if (path.exists() and exist_ok) or (not path.exists()):
        return str(path)
    else:
        dirs = glob.glob(f"{path}*")
        matches = [re.search(rf"%s(\d+)" % path.stem, d) for d in dirs]
        i = [int(m.groups()[0]) for m in matches if m]
        n = max(i) + 1 if i else 2
        return f"{path}{n}"

In [15]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [16]:
dataset_module = getattr(import_module("dataset"), 'MaskBaseDataset')

In [19]:
dataset = dataset_module(
        data_dir='/opt/ml/input/data/train/images',
    )

In [21]:
num_classes = dataset.num_classes

In [25]:
transform_module = getattr(import_module("dataset"), 'BaseAugmentation')  # default: BaseAugmentation
transform = transform_module(
    resize=(224,224),
    mean=dataset.mean,
    std=dataset.std,
)
dataset.set_transform(transform)

In [26]:
model_module = getattr(import_module("model"), 'ConvNextIn22ft1k')  # default: BaseModel
model = model_module(
    num_classes=num_classes
).to(device)
model = torch.nn.DataParallel(model)

In [27]:
train_set, val_set = dataset.split_dataset()

In [199]:
val_loader = DataLoader(
    val_set,
    batch_size=32,
    num_workers=multiprocessing.cpu_count()//2,
    shuffle=False,
    pin_memory=use_cuda,
    drop_last=True,
)

In [200]:
f1_pred = []
f1_labels = []

In [201]:
inputs, labels = next(iter(val_loader))

In [202]:
inputs = inputs.to(device)
labels = labels.to(device)

In [203]:
outs = model(inputs)

In [204]:
preds = torch.argmax(outs, dim = -1)

In [205]:
f1_pred += preds

In [206]:
f1_labels += labels

In [207]:
len(f1_pred)

32

In [208]:
len(f1_labels)

32

In [209]:
f1_pred

[tensor(5, device='cuda:0'),
 tensor(10, device='cuda:0'),
 tensor(16, device='cuda:0'),
 tensor(13, device='cuda:0'),
 tensor(10, device='cuda:0'),
 tensor(13, device='cuda:0'),
 tensor(11, device='cuda:0'),
 tensor(13, device='cuda:0'),
 tensor(15, device='cuda:0'),
 tensor(13, device='cuda:0'),
 tensor(5, device='cuda:0'),
 tensor(16, device='cuda:0'),
 tensor(13, device='cuda:0'),
 tensor(11, device='cuda:0'),
 tensor(15, device='cuda:0'),
 tensor(11, device='cuda:0'),
 tensor(5, device='cuda:0'),
 tensor(10, device='cuda:0'),
 tensor(12, device='cuda:0'),
 tensor(10, device='cuda:0'),
 tensor(16, device='cuda:0'),
 tensor(11, device='cuda:0'),
 tensor(10, device='cuda:0'),
 tensor(11, device='cuda:0'),
 tensor(10, device='cuda:0'),
 tensor(13, device='cuda:0'),
 tensor(13, device='cuda:0'),
 tensor(2, device='cuda:0'),
 tensor(5, device='cuda:0'),
 tensor(13, device='cuda:0'),
 tensor(7, device='cuda:0'),
 tensor(13, device='cuda:0')]

In [210]:
f1_labels

[tensor(4, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(4, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(4, device='cuda:0'),
 tensor(7, device='cuda:0'),
 tensor(4, device='cuda:0'),
 tensor(3, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(15, device='cuda:0'),
 tensor(4, device='cuda:0'),
 tensor(4, device='cuda:0'),
 tensor(13, device='cuda:0'),
 tensor(5, device='cuda:0'),
 tensor(4, device='cuda:0'),
 tensor(11, device='cuda:0'),
 tensor(10, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(3, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(2, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(10, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(3, device='cuda:0'),
 tensor(4, device='cuda:0'),
 tensor(14, device='cuda:0'),
 tensor(10, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(15, device='cuda:0'),
 tensor(3, device='cuda:0')]

In [211]:
f1_labels = torch.stack(f1_labels)
f1_labels

tensor([ 4,  1,  1,  4,  1,  4,  7,  4,  3,  0, 15,  4,  4, 13,  5,  4, 11, 10,
         1,  3,  0,  2,  1, 10,  0,  3,  4, 14, 10,  1, 15,  3],
       device='cuda:0')

In [212]:
f1_labels = f1_labels.cpu().numpy()

In [213]:
f1_labels

array([ 4,  1,  1,  4,  1,  4,  7,  4,  3,  0, 15,  4,  4, 13,  5,  4, 11,
       10,  1,  3,  0,  2,  1, 10,  0,  3,  4, 14, 10,  1, 15,  3])

In [214]:
f1_pred = torch.stack(f1_pred)
f1_pred = f1_pred.cpu().numpy()
f1_pred

array([ 5, 10, 16, 13, 10, 13, 11, 13, 15, 13,  5, 16, 13, 11, 15, 11,  5,
       10, 12, 10, 16, 11, 10, 11, 10, 13, 13,  2,  5, 13,  7, 13])

In [77]:
from sklearn.metrics import f1_score

In [215]:
f1_macro = f1_score(f1_labels, f1_pred, average='macro')

In [216]:
f1_macro

0.015873015873015872